## Some basic stuff

In [62]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np

In [4]:
def getSoup(link):
    sleep(0.1)
    r = requests.get(link)
    return BeautifulSoup(r.text,'lxml')

### Teams and their codes

#### Will be useful later on i think 

In [5]:
codes = getSoup('https://nhl.cz/sezona/tymy')

In [6]:
team_info_list = codes.findAll('a', {'class':'box-team__head'})
code_list = [x["href"] for x in team_info_list]
team_list = [h2.text for h2 in codes.findAll('h2')]
dict_team_codes = {team_list[i]: code_list[i] for i in range(len(code_list))}
dict_team_codes

{'Anaheim Ducks': '/klub/2004',
 'Arizona Coyotes': '/klub/2005',
 'Boston Bruins': '/klub/1987',
 'Buffalo Sabres': '/klub/1989',
 'Calgary Flames': '/klub/2006',
 'Carolina Hurricanes': '/klub/1990',
 'Colorado Avalanche': '/klub/2008',
 'Columbus Blue Jackets': '/klub/1991',
 'Dallas Stars': '/klub/2009',
 'Detroit Red Wings': '/klub/1992',
 'Edmonton Oilers': '/klub/2010',
 'Florida Panthers': '/klub/1993',
 'Chicago Blackhawks': '/klub/2007',
 'Los Angeles Kings': '/klub/2011',
 'Minnesota Wild': '/klub/2012',
 'Montreal Canadiens': '/klub/1994',
 'Nashville Predators': '/klub/2013',
 'New Jersey Devils': '/klub/1995',
 'New York Islanders': '/klub/1996',
 'New York Rangers': '/klub/1997',
 'Ottawa Senators': '/klub/1998',
 'Philadelphia Flyers': '/klub/1999',
 'Pittsburgh Penguins': '/klub/2000',
 'San Jose Sharks': '/klub/2014',
 'Seattle Kraken': '/klub/2730',
 'St. Louis Blues': '/klub/2015',
 'Tampa Bay Lightning': '/klub/2001',
 'Toronto Maple Leafs': '/klub/2002',
 'Vancouv

In [7]:
#sezony zatim rucne neprijde mi ze by to slo od nekud vycist
#2017 je 2017/2018 atd.
season_codes = {'2017': '6049', '2018': '6270', '2019': '6494', '2020': '6673','2021': '6733'}
seasons_used = ['2017', '2018', '2019', '2020','2021']

## How it works: one team and one team so far

## Sorry za mess, nektery variables jsou zbytecny, uklidim to pozdeji

##### Needs to be made to be more automated later
#### Later used: TOI = Team of Interest

In [8]:
team_of_interest = "Boston Bruins"
code_of_interest = dict_team_codes[team_of_interest]
season_of_interest = '2021'
code_of_season = season_codes[season_of_interest]
soup = getSoup(f'https://nhl.cz{code_of_interest}/zapasy?matchList-filter-season={season_of_interest}&matchList-filter-competition={code_of_season}')
#soup

In [9]:
all_games = soup.findAll('tr')

### Goals in a list

In [10]:
goals = [int(td.text) for td in soup.findAll('td', {'class':'preview__score'})]
goals[:5]

[3, 1, 6, 3, 1]

### Teams in a list

In [11]:
teams = [span.text for span in soup.findAll('span', {'class':'preview__name--long'})]
teams[:5]

['Boston Bruins',
 'Dallas Stars',
 'Philadelphia Flyers',
 'Boston Bruins',
 'Buffalo Sabres']

In [12]:
distinct_teams = []
for team in teams:
    if team not in distinct_teams:
        distinct_teams.append(team)

### Overtime or not

In [13]:
OT = []
for game in all_games:
    ot = game.find('span', {'class':'dot dot__blue'})
    if ot is None:
        OT.append(0)
    else:
        OT.append(1)
print(OT[:7])

[0, 0, 0, 0, 0, 0, 1]


### Game ID in a list

In [14]:
game_ids_list = soup.find_all("a", href=lambda value: value and value.startswith("/zapas/"))
game_ids = [x["href"] for x in game_ids_list][::2]
game_ids[:5]

['/zapas/2897006',
 '/zapas/2897032',
 '/zapas/2897045',
 '/zapas/2897063',
 '/zapas/2897081']

### Putting it in a dataframe

In [15]:
aux_dictionary = {'game_id':game_ids, 'team_home': teams[::2], 'team_away': teams[1::2],
    'goals_home': goals[::2], 'goals_away': goals[1::2], 'OT': OT} 
df = pd.DataFrame(aux_dictionary)
df.head()

,game_id,team_home,team_away,goals_home,goals_away,OT
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0


### Adding some more variables that might be useful

In [16]:
df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 3
df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 2
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 1
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 0

df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 0
df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 1
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 2
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 3

df.head()

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0,3.0,0.0
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0,3.0,0.0
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0,0.0,3.0
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0,3.0,0.0
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0,3.0,0.0


### Adding variable whether the team of interest is a home team

In [17]:
df.loc[df["team_home"] == team_of_interest, 'TOI_home'] = 1
df.loc[df["team_home"] != team_of_interest, 'TOI_home'] = 0

### Adding how many points TOI received

In [18]:
df.loc[(df["TOI_home"] == 1), 'TOI_points'] = df["points_home"]
df.loc[(df["TOI_home"] == 0), 'TOI_points'] = df["points_away"]

df.loc[(df["TOI_home"] == 1), 'TOI_goals_scored'] = df["goals_home"]
df.loc[(df["TOI_home"] == 0), 'TOI_goals_scored'] = df["goals_away"]
df.loc[(df["TOI_home"] == 1), 'TOI_goals_rec'] = df["goals_away"]
df.loc[(df["TOI_home"] == 0), 'TOI_goals_rec'] = df["goals_home"]

### Adding Output Variable: the result of our TOI

In [19]:
df.loc[df["OT"] == 1, 'TOI_result'] = "OT"
df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 3), 'TOI_result'] = "W"
df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 0), 'TOI_result'] = "L"
df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 3), 'TOI_result'] = "L"
df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 0), 'TOI_result'] = "W"

In [20]:
df.loc[(df["TOI_home"] == 1), 'other_team'] = df["team_away"]
df.loc[(df["TOI_home"] == 0), 'other_team'] = df["team_home"]
df.head()

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,Dallas Stars
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0,3.0,0.0,0.0,0.0,3.0,6.0,L,Philadelphia Flyers
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0,0.0,3.0,0.0,3.0,4.0,1.0,W,Buffalo Sabres
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0,3.0,0.0,1.0,3.0,4.0,3.0,W,San Jose Sharks
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0,3.0,0.0,0.0,0.0,1.0,4.0,L,Florida Panthers


### Adding variables - sum of last 20 games

In [23]:
TOI_points_tot = [None] * 20
TOI_goals_scored_tot = [None] * 20
TOI_goals_rec_tot = [None] * 20
for x in range(len(df)):
    if x > 19:
        TOI_points_tot.append(int(df[['TOI_points']][x-20:x].sum()))
        TOI_goals_scored_tot.append(int(df[['TOI_goals_scored']][x-20:x].sum()))
        TOI_goals_rec_tot.append(int(df[['TOI_goals_rec']][x-20:x].sum()))
df['TOI_points_tot'] = TOI_points_tot
df['TOI_goals_scored_tot'] = TOI_goals_scored_tot
df['TOI_goals_rec_tot'] = TOI_goals_rec_tot

df.iloc[19:25, 8:]

,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
19,0.0,3.0,2.0,0.0,W,Nashville Predators,NaN,NaN,NaN
20,1.0,1.0,2.0,3.0,OT,Tampa Bay Lightning,35.0,59.0,53.0
21,0.0,1.0,1.0,2.0,OT,Vancouver Canucks,33.0,58.0,55.0
22,0.0,3.0,3.0,2.0,W,Edmonton Oilers,34.0,56.0,51.0
23,0.0,3.0,4.0,2.0,W,Calgary Flames,34.0,55.0,52.0
24,1.0,0.0,1.0,4.0,L,Vegas Golden Knights,34.0,55.0,51.0


In [24]:
#tady jen neco zkousim
df[['TOI_home']][:20]
int(df[['TOI_points']][0:20].sum())

35

Takhle bude vypadat df, akorat v nim bude dat za vice sezon

## Creating the dataset later used for modellig

## ke kazdemu zapasu: 
- jeho id
- nazev druheho tymu
- vysledek TOI (output variable)
- jestli hraje TOI doma
- kolik ziskal TOI bodu v poslednich X (20/30) zapasech
- kolik dal TOI golu v poslednich X (20/30) zapasech
- kolik dostal TOI golu v poslednich X (20/30) zapasech

  
    - toto bude jeste vse potreba i pro druhy tym s kterym je prave ten zapas
    - tyhle radky plus radky o druhem tymu by uz meli jit nakrmit do modelu a to by melo byt vsechno

In [25]:
n_of_games = 20 #the number of previous games we will take into account, myslim ze 20 je tak optimalni

In [26]:
df_model = df[['game_id','TOI_result','other_team','TOI_home', 'TOI_points_tot', 'TOI_goals_scored_tot', 'TOI_goals_rec_tot']][20:].copy()
df_model

,game_id,TOI_result,other_team,TOI_home,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
20,/zapas/2897351,OT,Tampa Bay Lightning,1.0,35.0,59.0,53.0
21,/zapas/2897379,OT,Vancouver Canucks,0.0,33.0,58.0,55.0
22,/zapas/2897387,W,Edmonton Oilers,0.0,34.0,56.0,51.0
23,/zapas/2897403,W,Calgary Flames,0.0,34.0,55.0,52.0
24,/zapas/2897426,L,Vegas Golden Knights,1.0,34.0,55.0,51.0
...,...,...,...,...,...,...,...
77,/zapas/2898234,W,New York Rangers,1.0,34.0,63.0,54.0
78,/zapas/2897458,W,Montreal Canadiens,0.0,34.0,62.0,52.0
79,/zapas/2898259,W,Florida Panthers,1.0,34.0,64.0,53.0
80,/zapas/2898274,W,Buffalo Sabres,1.0,35.0,66.0,54.0


# Turning it into OOP

In [27]:
def create_link(team_of_interest,season_of_interest):
    code_of_season = season_codes[season_of_interest]
    code_of_interest = dict_team_codes[team_of_interest]
    return f'https://nhl.cz{code_of_interest}/zapasy?matchList-filter-season={season_of_interest}&matchList-filter-competition={code_of_season}'
create_link('Boston Bruins', '2021'),

('https://nhl.cz/klub/1987/zapasy?matchList-filter-season=2021&matchList-filter-competition=6733',)

In [28]:
#tady bude Class co idealne vrati df ktery pak pouzijem, muzem z toho vytisknout i nakej v prubehu tak jako KUK
class One_team_season:
   
    def __init__(self,team, season):
        self.team = team
        self.season = season
        
    def get_link(self):
        code_of_season = season_codes[self.season]
        code_of_interest = dict_team_codes[self.team]
        return f'https://nhl.cz{code_of_interest}/zapasy?matchList-filter-season={self.season}&matchList-filter-competition={code_of_season}'
    
    def get_soup(self):
        r = requests.get(self.get_link())
        if r.status_code == 200:
            r.encoding='UTF-8'
            return BeautifulSoup(r.text)
        else:
            raise Exception(f'Cannot access the url. Returning status_code {r.status_code}')
            
    def get_goals(self,soup):
        return [int(td.text) for td in soup.findAll('td', {'class':'preview__score'})]
    
    def get_teams(self,soup):
        return [span.text for span in soup.findAll('span', {'class':'preview__name--long'})]
    
    def get_OT(self,soup):
        all_games = soup.findAll('tr')
        OT = []
        for game in all_games:
            ot = game.find('span', {'class':'dot dot__blue'})
            if ot is None:
                OT.append(0)
            else:
                OT.append(1)
        return OT
    
    def get_game_IDs(self,soup):
        game_ids_list = soup.find_all("a", href=lambda value: value and value.startswith("/zapas/"))
        game_ids = [x["href"] for x in game_ids_list][::2]
        return game_ids
    
    def create_df(self):
        soup = self.get_soup()
        goals = self.get_goals(soup)
        teams = self.get_teams(soup)
        OT = self.get_OT(soup)
        game_ids = self.get_game_IDs(soup)
        
        #covid vsuvka - apparently games cancelled due to covid were left blank and would cause errors
        if len(goals)/2 == len(teams)/2 == len(OT) == len(game_ids):
            pass
        else:
            teams = teams[:len(goals)]
            OT = OT[:int(len(goals)/2)]
            game_ids = game_ids[:int(len(goals)/2)]
        #konec covid vsuvky

        
        aux_dictionary = {'game_id':game_ids, 'team_home': teams[::2], 'team_away': teams[1::2],
        'goals_home': goals[::2], 'goals_away': goals[1::2], 'OT': OT} 
        df = pd.DataFrame(aux_dictionary)
        return df
    
    def create_model_df(self):
        df = self.create_df()
        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 3
        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 2
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 1
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 0

        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 0
        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 1
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 2
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 3
        
        df.loc[df["team_home"] == self.team, 'TOI_home'] = 1
        df.loc[df["team_home"] != self.team, 'TOI_home'] = 0
        
        df.loc[(df["TOI_home"] == 1), 'TOI_points'] = df["points_home"]
        df.loc[(df["TOI_home"] == 0), 'TOI_points'] = df["points_away"]

        df.loc[(df["TOI_home"] == 1), 'TOI_goals_scored'] = df["goals_home"]
        df.loc[(df["TOI_home"] == 0), 'TOI_goals_scored'] = df["goals_away"]
        df.loc[(df["TOI_home"] == 1), 'TOI_goals_rec'] = df["goals_away"]
        df.loc[(df["TOI_home"] == 0), 'TOI_goals_rec'] = df["goals_home"]
        
        df.loc[df["OT"] == 1, 'TOI_result'] = "OT"
        df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 3), 'TOI_result'] = "W"
        df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 0), 'TOI_result'] = "L"
        df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 3), 'TOI_result'] = "L"
        df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 0), 'TOI_result'] = "W"
        
        df.loc[(df["TOI_home"] == 1), 'other_team'] = df["team_away"]
        df.loc[(df["TOI_home"] == 0), 'other_team'] = df["team_home"]
        
        df.loc[(df["TOI_home"] == 1), 'TOI'] = df["team_home"]
        df.loc[(df["TOI_home"] == 0), 'TOI'] = df["team_away"]
        
        return df


    

## Joining DataFrames for multiple seasons

In [30]:
#trva to asi 2 minuty, coz neni ZAS TAK hrozny, ale jeste se kouknu jestli to nepujde nejak zrychlit
individual_teams = []
for team in distinct_teams:
    one_team = []
    for season in seasons_used:
        one_team.append(One_team_season(team, season).create_model_df())
    one_team = pd.concat(one_team, ignore_index=True)
    individual_teams.append(one_team)

individual_teams[1] #looks promising

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team,TOI
0,/zapas/2526069,Dallas Stars,Vegas Golden Knights,1,2,0,0.0,3.0,1.0,0.0,1.0,2.0,L,Vegas Golden Knights,Dallas Stars
1,/zapas/2526078,St. Louis Blues,Dallas Stars,4,2,0,3.0,0.0,0.0,0.0,2.0,4.0,L,St. Louis Blues,Dallas Stars
2,/zapas/2526097,Dallas Stars,Detroit Red Wings,4,2,0,3.0,0.0,1.0,3.0,4.0,2.0,W,Detroit Red Wings,Dallas Stars
3,/zapas/2526107,Nashville Predators,Dallas Stars,4,1,0,3.0,0.0,0.0,0.0,1.0,4.0,L,Nashville Predators,Dallas Stars
4,/zapas/2526123,Dallas Stars,Colorado Avalanche,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,Colorado Avalanche,Dallas Stars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,/zapas/2898216,Calgary Flames,Dallas Stars,4,2,0,3.0,0.0,0.0,0.0,2.0,4.0,L,Calgary Flames,Dallas Stars
367,/zapas/2898233,Dallas Stars,Seattle Kraken,3,2,0,3.0,0.0,1.0,3.0,3.0,2.0,W,Seattle Kraken,Dallas Stars
368,/zapas/2898257,Dallas Stars,Vegas Golden Knights,3,2,1,2.0,1.0,1.0,2.0,3.0,2.0,OT,Vegas Golden Knights,Dallas Stars
369,/zapas/2898269,Dallas Stars,Arizona Coyotes,3,4,1,1.0,2.0,1.0,1.0,3.0,4.0,OT,Arizona Coyotes,Dallas Stars


In [38]:
def create_final_df(df):
    TOI_points_tot = [None] * 20
    TOI_goals_scored_tot = [None] * 20
    TOI_goals_rec_tot = [None] * 20
    for x in range(len(df)):
        if x > 19:
            TOI_points_tot.append(int(df[['TOI_points']][x-20:x].sum()))
            TOI_goals_scored_tot.append(int(df[['TOI_goals_scored']][x-20:x].sum()))
            TOI_goals_rec_tot.append(int(df[['TOI_goals_rec']][x-20:x].sum()))
    df['TOI_points_tot'] = TOI_points_tot
    df['TOI_goals_scored_tot'] = TOI_goals_scored_tot
    df['TOI_goals_rec_tot'] = TOI_goals_rec_tot
    return df[['game_id', 'TOI_result', 'TOI_home','TOI', 'other_team', 'TOI_points_tot',
               'TOI_goals_scored_tot', 'TOI_goals_rec_tot']][20:]

In [54]:
#dale:
#zkontrolovat jestli to funguje a dela to to, co to ma delat
#asi bude nejlepsi kdyz se takovej dataset udela pro vsech 32 tymu protoze ty data pak budou potreba na training

'Philadelphia Flyers'

In [40]:
final_dfs = []
for team in individual_teams:
    final = create_final_df(team)
    final_dfs.append(final)



In [47]:
final_dfs[14].head()

,game_id,TOI_result,TOI_home,TOI,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
20,/zapas/2526361,W,1.0,New York Rangers,Ottawa Senators,27.0,63.0,67.0
21,/zapas/2526380,W,0.0,New York Rangers,Carolina Hurricanes,30.0,64.0,63.0
22,/zapas/2526395,OT,1.0,New York Rangers,Detroit Red Wings,33.0,65.0,56.0
23,/zapas/2526414,OT,1.0,New York Rangers,Vancouver Canucks,32.0,65.0,57.0
24,/zapas/2526423,L,1.0,New York Rangers,Florida Panthers,34.0,68.0,57.0


In [50]:
final_dfs[20].head()

,game_id,TOI_result,TOI_home,TOI,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
20,/zapas/2526373,W,0.0,Minnesota Wild,Buffalo Sabres,30.0,59.0,55.0
21,/zapas/2526390,OT,1.0,Minnesota Wild,Colorado Avalanche,33.0,62.0,55.0
22,/zapas/2526409,L,0.0,Minnesota Wild,St. Louis Blues,34.0,61.0,52.0
23,/zapas/2526419,L,0.0,Minnesota Wild,Winnipeg Jets,31.0,59.0,56.0
24,/zapas/2526438,W,1.0,Minnesota Wild,Vegas Golden Knights,30.0,57.0,58.0


### Calculating Home Advantage

In [31]:
sum_wins_home = sum(df["goals_home"] > df["goals_away"])
sum_wins_away = sum(df["goals_home"] < df["goals_away"])
 
home_winrate = sum_wins_home/(sum_wins_home+sum_wins_away)
home_winrate

0.5121951219512195

### List of all the teams

### Neděláme to zbytečně dvakrát jednou už distinct_teams definujem výše a i to výše používáme  

In [32]:
distinct_teams = []
for team in teams:
    if team not in distinct_teams:
        distinct_teams.append(team)
print(distinct_teams)

['Boston Bruins', 'Dallas Stars', 'Philadelphia Flyers', 'Buffalo Sabres', 'San Jose Sharks', 'Florida Panthers', 'Carolina Hurricanes', 'Detroit Red Wings', 'Toronto Maple Leafs', 'Ottawa Senators', 'Edmonton Oilers', 'New Jersey Devils', 'Montreal Canadiens', 'Calgary Flames', 'New York Rangers', 'Vancouver Canucks', 'Nashville Predators', 'Tampa Bay Lightning', 'Vegas Golden Knights', 'New York Islanders', 'Minnesota Wild', 'Washington Capitals', 'Winnipeg Jets', 'Anaheim Ducks', 'Colorado Avalanche', 'Arizona Coyotes', 'Seattle Kraken', 'Pittsburgh Penguins', 'Los Angeles Kings', 'Columbus Blue Jackets', 'Chicago Blackhawks', 'St. Louis Blues']


In [157]:
df

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,Dallas Stars,NaN,NaN,NaN
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0,3.0,0.0,0.0,0.0,3.0,6.0,L,Philadelphia Flyers,NaN,NaN,NaN
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0,0.0,3.0,0.0,3.0,4.0,1.0,W,Buffalo Sabres,NaN,NaN,NaN
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0,3.0,0.0,1.0,3.0,4.0,3.0,W,San Jose Sharks,NaN,NaN,NaN
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0,3.0,0.0,0.0,0.0,1.0,4.0,L,Florida Panthers,NaN,NaN,NaN
5,/zapas/2897091,Carolina Hurricanes,Boston Bruins,3,0,0,3.0,0.0,0.0,0.0,0.0,3.0,L,Carolina Hurricanes,NaN,NaN,NaN
6,/zapas/2897105,Boston Bruins,Florida Panthers,3,2,1,2.0,1.0,1.0,2.0,3.0,2.0,OT,Florida Panthers,NaN,NaN,NaN
7,/zapas/2897132,Boston Bruins,Detroit Red Wings,5,1,0,3.0,0.0,1.0,3.0,5.0,1.0,W,Detroit Red Wings,NaN,NaN,NaN
8,/zapas/2897148,Toronto Maple Leafs,Boston Bruins,5,2,0,3.0,0.0,0.0,0.0,2.0,5.0,L,Toronto Maple Leafs,NaN,NaN,NaN
9,/zapas/2897168,Boston Bruins,Ottawa Senators,3,2,0,3.0,0.0,1.0,3.0,3.0,2.0,W,Ottawa Senators,NaN,NaN,NaN


### After picking 2 teams both their final_dfs are found, saved and displayed

In [137]:
team_1 = "New York Rangers"
team_2 = "Philadelphia Flyers"

In [138]:
team_1_index = distinct_teams.index(team_1)
team_2_index = distinct_teams.index(team_2)
team_1_index

14

In [142]:
matches_team_1 = final_dfs[team_1_index]
matches_team_1

,game_id,TOI_result,TOI_home,TOI,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
20,/zapas/2526361,W,1.0,New York Rangers,Ottawa Senators,27.0,63.0,67.0
21,/zapas/2526380,W,0.0,New York Rangers,Carolina Hurricanes,30.0,64.0,63.0
22,/zapas/2526395,OT,1.0,New York Rangers,Detroit Red Wings,33.0,65.0,56.0
23,/zapas/2526414,OT,1.0,New York Rangers,Vancouver Canucks,32.0,65.0,57.0
24,/zapas/2526423,L,1.0,New York Rangers,Florida Panthers,34.0,68.0,57.0
...,...,...,...,...,...,...,...,...
367,/zapas/2898222,W,0.0,New York Rangers,New York Islanders,40.0,67.0,47.0
368,/zapas/2898234,L,0.0,New York Rangers,Boston Bruins,43.0,71.0,44.0
369,/zapas/2898264,L,1.0,New York Rangers,Carolina Hurricanes,40.0,65.0,43.0
370,/zapas/2897488,L,1.0,New York Rangers,Montreal Canadiens,38.0,64.0,44.0


In [141]:
matches_team_2 = final_dfs[team_2_index]
matches_team_2

,game_id,TOI_result,TOI_home,TOI,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
20,/zapas/2526370,L,1.0,Philadelphia Flyers,Vancouver Canucks,27.0,56.0,56.0
21,/zapas/2526377,OT,0.0,Philadelphia Flyers,New York Islanders,24.0,53.0,58.0
22,/zapas/2526391,OT,1.0,Philadelphia Flyers,New York Islanders,25.0,56.0,60.0
23,/zapas/2526417,OT,0.0,Philadelphia Flyers,Pittsburgh Penguins,24.0,57.0,63.0
24,/zapas/2526424,L,1.0,Philadelphia Flyers,San Jose Sharks,25.0,56.0,62.0
...,...,...,...,...,...,...,...,...
366,/zapas/2898218,W,0.0,Philadelphia Flyers,Montreal Canadiens,15.0,54.0,84.0
367,/zapas/2898249,W,1.0,Philadelphia Flyers,Pittsburgh Penguins,18.0,59.0,84.0
368,/zapas/2898252,L,0.0,Philadelphia Flyers,Chicago Blackhawks,20.0,60.0,81.0
369,/zapas/2898270,L,0.0,Philadelphia Flyers,Winnipeg Jets,17.0,56.0,80.0


In [156]:
matches_between_them = matches_team_1.merge(matches_team_2, left_on='game_id', right_on='game_id',suffixes=('', '2'))
matches_between_them.drop(columns=['TOI_result2', 'TOI_home2','TOI2','other_team2','TOI_points_tot2','TOI_goals_scored_tot2','TOI_goals_rec_tot2'], inplace=True)
pd.set_option("display.max_rows", None, "display.max_columns", None)
matches_between_them


,game_id,TOI_result,TOI_home,TOI,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
0,/zapas/2526742,W,1.0,New York Rangers,Philadelphia Flyers,26.0,54.0,57.0
1,/zapas/2526959,L,1.0,New York Rangers,Philadelphia Flyers,17.0,46.0,75.0
2,/zapas/2527191,L,0.0,New York Rangers,Philadelphia Flyers,22.0,55.0,74.0
3,/zapas/2527311,L,0.0,New York Rangers,Philadelphia Flyers,23.0,57.0,69.0
4,/zapas/2647530,L,0.0,New York Rangers,Philadelphia Flyers,33.0,65.0,59.0
5,/zapas/2647760,OT,1.0,New York Rangers,Philadelphia Flyers,29.0,62.0,68.0
6,/zapas/2647972,L,1.0,New York Rangers,Philadelphia Flyers,27.0,59.0,76.0
7,/zapas/2648413,W,0.0,New York Rangers,Philadelphia Flyers,19.0,50.0,70.0
8,/zapas/2806950,L,0.0,New York Rangers,Philadelphia Flyers,30.0,62.0,62.0
9,/zapas/2807370,L,0.0,New York Rangers,Philadelphia Flyers,40.0,68.0,49.0


In [ ]:
pd.set_option("display.max_rows", 15, "display.max_columns", 15) #just so we dont display so many rows next time


In [161]:
matches_between_them_ids = matches_between_them['game_id']
matches_between_them_ids

0     /zapas/2526742
1     /zapas/2526959
2     /zapas/2527191
3     /zapas/2527311
4     /zapas/2647530
5     /zapas/2647760
6     /zapas/2647972
7     /zapas/2648413
8     /zapas/2806950
9     /zapas/2807370
10    /zapas/2807387
11    /zapas/2893477
12    /zapas/2893518
13    /zapas/2893656
14    /zapas/2893672
15    /zapas/2893728
16    /zapas/2893743
17    /zapas/2893463
18    /zapas/2893949
19    /zapas/2897332
20    /zapas/2897659
21    /zapas/2898088
22    /zapas/2898160
Name: game_id, dtype: object

### Kolik bodů získal tým celkem

In [34]:
#tady je problem že je to zatím pro jeden tým určený tím boston.
#Až budem vyjíždět týmy je třeba upravit
toi=boston.create_model_df()
points_home = toi.groupby(['team_home'])['points_home'].sum().reset_index()
points_away = toi.groupby(['team_away'])['points_away'].sum().reset_index()
points_home.rename(columns = {'team_home':'team'}, inplace = True)
points_away.rename(columns = {'team_away':'team'}, inplace = True)
tp = points_home.merge(points_away, left_on='team', right_on='team')
total_poins = tp["points_home"] + tp["points_away"]
tp["points_total"] = points_total
tp.drop(tp[tp.points_total < 15].index, inplace=True)
if len(tp) != 1:
            raise Exception('There is more than 1 team left')
tp

NameError: name 'boston' is not defined

In [ ]:
if len(tp) != 1:
            raise Exception('There is more than 1 team left')


,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team,TOI,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
0,/zapas/2526069,Dallas Stars,Vegas Golden Knights,1,2,0,0.0,3.0,1.0,0.0,1.0,2.0,L,Vegas Golden Knights,Dallas Stars,NaN,NaN,NaN
1,/zapas/2526078,St. Louis Blues,Dallas Stars,4,2,0,3.0,0.0,0.0,0.0,2.0,4.0,L,St. Louis Blues,Dallas Stars,NaN,NaN,NaN
2,/zapas/2526097,Dallas Stars,Detroit Red Wings,4,2,0,3.0,0.0,1.0,3.0,4.0,2.0,W,Detroit Red Wings,Dallas Stars,NaN,NaN,NaN
3,/zapas/2526107,Nashville Predators,Dallas Stars,4,1,0,3.0,0.0,0.0,0.0,1.0,4.0,L,Nashville Predators,Dallas Stars,NaN,NaN,NaN
4,/zapas/2526123,Dallas Stars,Colorado Avalanche,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,Colorado Avalanche,Dallas Stars,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,/zapas/2898216,Calgary Flames,Dallas Stars,4,2,0,3.0,0.0,0.0,0.0,2.0,4.0,L,Calgary Flames,Dallas Stars,32.0,53.0,63.0
367,/zapas/2898233,Dallas Stars,Seattle Kraken,3,2,0,3.0,0.0,1.0,3.0,3.0,2.0,W,Seattle Kraken,Dallas Stars,32.0,55.0,63.0
368,/zapas/2898257,Dallas Stars,Vegas Golden Knights,3,2,1,2.0,1.0,1.0,2.0,3.0,2.0,OT,Vegas Golden Knights,Dallas Stars,33.0,54.0,62.0
369,/zapas/2898269,Dallas Stars,Arizona Coyotes,3,4,1,1.0,2.0,1.0,1.0,3.0,4.0,OT,Arizona Coyotes,Dallas Stars,35.0,55.0,60.0
